In [1]:
#change working directory
import os
print(os.getcwd())
os.chdir('/home/student/A2/drive-download-20240507T073745Z-001')


from datetime import datetime, timedelta
import pymongo
from pymongo import MongoClient
#Residence wifi ip address: 172.16.62.212
ipadd = "172.16.33.120"


client = MongoClient(ipadd, 27017)
db = client.fit3182_db
collection = db.fit3182_assignment_db

#res = collection.find().sort('surface_temperature_celcius', pymongo.DESCENDING).limit(10)
res = collection.find().sort('date', pymongo.DESCENDING).limit(1)

for d in res:
    res = d
#     print(d)

latest_date = res['date']
#string to datetime object
# latest_date = datetime.strptime(latest_date, '%Y-%m-%d %H:%M:%S')
#string to datetime object
latest_date = datetime.strptime(latest_date, '%d/%m/%Y')
start_date = latest_date + timedelta(days=1)
print(start_date)

/home/student/A2
2024-01-01 00:00:00


In [2]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import json
import pandas as pd

hostip = ipadd # change it to your IP

def connect_kafka_producer():
    producer = None
    
    try:
        producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],api_version=(0,10))
    
    except Exception as ex:
        print("Exception while connecting Kafka")
        print(str(ex))
    finally:
        return producer 
    
def publish_msg(producer, topic, data):
    try:
        #print(data)        #send the  value by dumps to JSON object
                            #then encode with utf-8 into bytes
        value_bytes = json.dumps(data).encode('utf-8')
        producer.send(topic, value=value_bytes)
        #print(data)
        """
        The flush() method in Kafka’s producer API is used to ensure that all 
        previously sent messages have been transmitted to the server and acknowledged 
        before proceeding. When you call flush(), it blocks the current thread and waits 
        for the producer to complete the sending of all records
        """
        producer.flush()  
    except Exception as ex:
        print('Exception from publish_msg func')
        print(str(ex))
    


In [3]:
"""
Event Producer 2
"""
hotspot_AQUA = pd.read_csv("./hotspot_AQUA_streaming.csv", sep=',')

hotspot_AQUA.head()


"""
Main Function
"""
if __name__ == '__main__':
    topic = 'Producer2'
    print('Publishing records..')
    producer02 = connect_kafka_producer()
    
    
    rand_row_indices = list(range(len(hotspot_AQUA.index)))
    random.shuffle(rand_row_indices) 
    # random choose time from 0 to 10
    max_time = 2 #test
    #rand_time = list(range(1,max_time))
#     random.shuffle(rand_time) 
     

    for index, hotspot_AQUA_row in hotspot_AQUA.iterrows():
        hotspot_AQUA_row = hotspot_AQUA.loc[rand_row_indices[index]]

        hotspot_AQUA_doc = {
            'latitude': hotspot_AQUA_row['latitude'],
            'longitude': hotspot_AQUA_row['longitude'],
            'confidence': hotspot_AQUA_row['confidence'],
            'surface_temperature_celcius': hotspot_AQUA_row['surface_temperature_celcius'],
            'producer': "Producer_2",
            'random_time': rand_row_indices[index]%max_time + 1 # at least 1 sec
        }
        #update start date
        #print(climate_doc['date'])
        #producer publish msg in strictured streaming
        # convert the dictionary to JSON-formatted string and encode as UFT-8 by publish_msg
        # msg = json.dumps(climate_doc)
        #print(climate_doc)
        print(hotspot_AQUA_doc['random_time'])
        publish_msg(producer02, topic, hotspot_AQUA_doc)
        # every random seconds
        sleep(hotspot_AQUA_doc['random_time']) 



Publishing records..
2
2
2
1
1
1
2
2
1
1
2
2
2
2
2
1
1
2
2
1
1
2
2
2
2
1
1
1
1
1
1
2
1
2
2
1
2
2
2
2
1
2
2
2
2
1
1
2
1
2
2
2
1
2
2
2
1
1
2
2
1
1
1
2
1
2
1
1
1
1
1
2
2
1
2
2
1
2
2
1
1
1
2
1
2
2
1
2
1
2
2
2
2
2
1
2
1
1
1
2
1
1
1
2
1
2
1
2
2
1
2
2
2
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
2
2
1
2
2
1
1
2
2
1
2
1
1
2
2
2
1
1
1
1
1
2
2
2
1
2
1
1
2
2
2
1
2
1
1
1
1
2
2
2
1
1
2
2
2
2
2
2
2
2
1
1
1
2
1
1
1
2
2
1
1
2
2
1
2
1
1
1
2
2
1
1
2
1
2
2
1
2
1
2
2
1
2
2
2
2
2
1
1
2
1
2
2
2
1
2
1
1
1
2
2
1
2
2
1
2
2
1
2
1
2
2
2
1
2
2
1
1
1
2
1
2
1
1
1
2
1
1
1
2
1
2
1
2
2
2
1
2
1
1
2
2
1
2
2
1
2
1
1
1
1
1
1
1
2
2
2
2
1
1
2
1
1
1
2
2
1
2
2
2
1
2
1
1
1
2
1
1
1
1
2
2
1
2
2
2
2
1
2
2
1
1
1
2
2
2
2
1
1
2
2
1
2
1
1
1
1
1
2
2
2
1
2
2
2
1
1
1
2
2
1
2
2
2
2
2
2
1
2
2
1
2
2
2
2
1
2
1
2
1
2
2
1
1
2
2
2
1
1
1
1
1
1
2
2
2
2
1
1
1
1
2
1
2
1
1
1
2
1
1
1
1
2
2
1
1
1
2
2
2
1
2
2
2
1
1
2
1
2
1
2
2
2
2
1
2
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
2
2
1
2
1
1
1
2
1
1
2
2
2
1
2
2
1
2
2
1
1
1
2
1
1
1
1
1
2
2
2
2
2
2
1
1
1
2
2
2
2
1
2
1

Node 1 connection failed -- refreshing metadata


1


Node 1 connection failed -- refreshing metadata


2


...

1


Node 1 connection failed -- refreshing metadata


1


Node 1 connection failed -- refreshing metadata


KeyboardInterrupt: 